In [1]:
import os

In [2]:
%pwd

'd:\\Chest-Cancer-Classification-using-DVC-and-MLflow\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Chest-Cancer-Classification-using-DVC-and-MLflow'

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int    

In [9]:
from Chest_cancer_classification.constants import *
from Chest_cancer_classification.utils.common import save_json,read_yaml,create_directories
 

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest_ct_Scan_data/data",
            mlflow_uri=os.environ["MLFLOW_TRACKING_URI"],
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()

        
        self.score = self.model.evaluate(self.valid_generator, verbose=1)

        
        y_true = self.valid_generator.classes
        y_pred_probs = self.model.predict(self.valid_generator, verbose=1)
        y_pred = np.argmax(y_pred_probs, axis=1)

        
        auc = roc_auc_score(y_true, y_pred_probs[:, 1]) if y_pred_probs.shape[1] == 2 else None
        precision = precision_score(y_true, y_pred, average="binary")
        recall = recall_score(y_true, y_pred, average="binary")
        f1 = f1_score(y_true, y_pred, average="binary")

        self.metrics = {
            "loss": self.score[0],
            "accuracy": self.score[1],
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "auc": auc
        }

        self.save_score()

    def save_score(self):
        scores = self.metrics
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                self.metrics
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-08-09 20:36:10,575:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-08-09 20:36:10,587:INFO:common:yaml file: params.yaml loaded successfully]
[2025-08-09 20:36:10,589:INFO:common:created directory at: artifacts]
Found 144 images belonging to 2 classes.
9/9 [==============================] - 11s 1s/step
[2025-08-09 20:36:34,960:INFO:common:json file saved at: scores.json]


2025/08/09 20:36:36 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-09 20:36:38,057:WARNING:save:Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmpljpokxu0\model\data\model\assets
[2025-08-09 20:36:38,862:INFO:builder_impl:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmpljpokxu0\model\data\model\assets]


d:\Chest-Cancer-Classification-using-DVC-and-MLflow\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/08/09 20:37:47 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
